In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('Breast Cancer Original (1).csv')

In [3]:
df

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,diagnosis
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890,M
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902,M
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758,M
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300,M
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,M
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,M
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,M
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,M


In [4]:
#replacing M and B with 1 and 0
df['diagnosis'] = df['diagnosis'].replace({'M': 1, 'B': 0})

C:\Users\balij\AppData\Local\Temp\ipykernel_15548\1601235559.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['diagnosis'] = df['diagnosis'].replace({'M': 1, 'B': 0})


In [4]:
df.isna().sum()

radius_mean                0
texture_mean               0
perimeter_mean             0
area_mean                  0
smoothness_mean            0
compactness_mean           0
concavity_mean             0
concave points_mean        0
symmetry_mean              0
fractal_dimension_mean     0
radius_se                  0
texture_se                 0
perimeter_se               0
area_se                    0
smoothness_se              0
compactness_se             0
concavity_se               0
concave points_se          0
symmetry_se                0
fractal_dimension_se       0
radius_worst               0
texture_worst              0
perimeter_worst            0
area_worst                 0
smoothness_worst           0
compactness_worst          0
concavity_worst            0
concave points_worst       0
symmetry_worst             0
fractal_dimension_worst    0
diagnosis                  0
dtype: int64

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve, validation_curve
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

class ModelAnalyzer:
    def __init__(self, X, y, models):
        self.X = X
        self.y = y
        self.models = models
        self.best_models = {}
        self.scaler = StandardScaler()
        self.X_scaled = self.scaler.fit_transform(X)
        
    def perform_grid_search(self):
        """Perform grid search for all models"""
        for name, (model, params) in self.models.items():
            grid_search = GridSearchCV(model, params, cv=5, scoring='accuracy')
            grid_search.fit(self.X_scaled, self.y)
            self.best_models[name] = grid_search.best_estimator_
            print(f"\n{name} Best Parameters:", grid_search.best_params_)
            print(f"{name} Best Score:", grid_search.best_score_)
            
    def plot_learning_curves(self, name, model):
        """Plot learning curves to analyze overfitting/underfitting"""
        train_sizes = np.linspace(0.1, 1.0, 10)
        train_sizes, train_scores, val_scores = learning_curve(
            model, self.X_scaled, self.y, train_sizes=train_sizes, cv=5)
        
        train_mean = np.mean(train_scores, axis=1)
        train_std = np.std(train_scores, axis=1)
        val_mean = np.mean(val_scores, axis=1)
        val_std = np.std(val_scores, axis=1)
        
        plt.figure(figsize=(10, 6))
        plt.plot(train_sizes, train_mean, label='Training score')
        plt.plot(train_sizes, val_mean, label='Cross-validation score')
        plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.1)
        plt.fill_between(train_sizes, val_mean - val_std, val_mean + val_std, alpha=0.1)
        plt.xlabel('Training Examples')
        plt.ylabel('Score')
        plt.title(f'Learning Curves for {name}')
        plt.legend(loc='best')
        plt.grid()
        plt.show()
        
    def plot_misclassification_matrix(self, name, model):
        """Plot confusion matrix for misclassification analysis"""
        X_train, X_test, y_train, y_test = train_test_split(self.X_scaled, self.y, test_size=0.2)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        cm = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
        plt.title(f'Confusion Matrix for {name}')
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        plt.show()
        
    def analyze_model_complexity(self, name, model):
        """Analyze model complexity based on parameters"""
        if hasattr(model, 'n_estimators'):
            print(f"\n{name} Complexity:")
            print(f"Number of estimators: {model.n_estimators}")
        if hasattr(model, 'max_depth'):
            print(f"Max depth: {model.max_depth}")
        if name == "SVM":
            print(f"Kernel: {model.kernel}")
            print(f"C parameter: {model.C}")
        if name == "KNN":
            print(f"Number of neighbors: {model.n_neighbors}")
            
    def convergence_test(self, name, model):
        """Test model convergence with increasing iterations"""
        X_train, X_test, y_train, y_test = train_test_split(self.X_scaled, self.y, test_size=0.2)
        
        if hasattr(model, 'n_estimators'):
            n_estimators_range = np.arange(10, 210, 20)
            train_scores = []
            test_scores = []
            
            for n_estimators in n_estimators_range:
                model.set_params(n_estimators=n_estimators)
                model.fit(X_train, y_train)
                train_scores.append(accuracy_score(y_train, model.predict(X_train)))
                test_scores.append(accuracy_score(y_test, model.predict(X_test)))
            
            plt.figure(figsize=(10, 6))
            plt.plot(n_estimators_range, train_scores, label='Training Score')
            plt.plot(n_estimators_range, test_scores, label='Testing Score')
            plt.xlabel('Number of Estimators')
            plt.ylabel('Accuracy')
            plt.title(f'Convergence Test for {name}')
            plt.legend()
            plt.grid()
            plt.show()
    # Define Models and Hyperparameter Grids
models = {
    "Random Forest": (RandomForestClassifier(), {"n_estimators": [100, 200], "max_depth": [5, 10]}),
    "Gradient Boosting": (GradientBoostingClassifier(), {"n_estimators": [100, 200], "learning_rate": [0.01, 0.1]}),
    "Bagging": (BaggingClassifier(), {"n_estimators": [10, 50]}),
    "XGBoost": (XGBClassifier(), {"n_estimators": [100, 200], "learning_rate": [0.01, 0.1], "max_depth": [3, 5]}),
    "KNN": (KNeighborsClassifier(), {"n_neighbors": [3, 5, 7]}),
    "Logistic Regression": (LogisticRegression(), {"C": [0.1, 1, 10]}),
    "SVM": (SVC(), {"C": [0.1, 1, 10], "kernel": ["linear", "rbf"]})
}

    def run_complete_analysis(self):
        """Run complete analysis for all models"""
        self.perform_grid_search()
        
        for name, model in self.best_models.items():
            print(f"\n{'='*50}")
            print(f"Analysis for {name}")
            print('='*50)
            
            # Learning curves
            self.plot_learning_curves(name, model)
            
            # Misclassification matrix
            self.plot_misclassification_matrix(name, model)
            
            # Model complexity
            self.analyze_model_complexity(name, model)
            
            # Convergence test
            self.convergence_test(name, model)

# Usage example:
# Assuming you have your X and y data ready
analyzer = ModelAnalyzer(df.drop(columns=['diagnosis']), df['diagnosis'], models)
analyzer.run_complete_analysis()


NameError: name 'models' is not defined